In [4]:
library("lme4")
library("ggplot2")
library("dplyr")
library(MASS)
library("BayesFactor")
library("car")
library("scales")
library("lmerTest")
library("MuMIn")
library("plyr")
library("rstatix")
library("ggpubr")
library("knitr")
library("corrplot")
library("RColorBrewer")

In [5]:
#Load the data => Here, we are filtering the cases in which subjects had a MuIFI of 0
FinalPath<-'./R_Data_ALL.txt'
data <- read.delim(FinalPath, sep="\t", header=T, stringsAsFactors=F)

In [6]:
table(data$Agent, data$GameNb)

                   
                      1   2   3   4   5   6   7   8   9  10  11  12  13  14  15
  Coupled           100 100 100 100 100 100 100 100 100 100 100 100 100 100 100
  Coupled_Strategy   32  32  32  32  32  32  32  32  32  32  32  32  32  32  32
  MS17              100 100 100 100 100 100 100 100 100 100 100 100 100 100 100
  Streamed          100 100 100 100 100 100 100 100 100 100 100 100 100 100 100
  Streamed_Strategy   9   9   9   9   9   9   9   9   9   9   9   9   9   9   9

# Compute BIC for fast-shooting model

In [7]:
#RMSE function definition
RMSE = function(mod, obs){
  sqrt(mean((mod - obs)^2))
}

RSS = function(mod,obs){
    sum((mod - obs)^2)
}

BIC = function(RSS,n,k){
    n*log(RSS/n)+k*log(n)
}
#AIC = 2k + nln(RSS)
AIC = function(RSS,n,k){
    2*k + n*log(RSS)
}

In [8]:
#Filtered data set
fast_data <- data %>% filter(!Agent=='Coupled_Strategy')
head(fast_data,10)

,Agent,subject,GameNb,Deflations,Misses,Resets,ShotReg,Entropy,LogCV,ShotPer,RightReg,Score
,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,Coupled,model_6592528450_1,1,96,63,3,0.2749406,2.938219,-0.9013436,480,0.1648576,256
2,Coupled,model_6592528450_1,2,22,107,2,0.4118143,2.848877,-1.3099846,512,0.3605711,888
3,Coupled,model_6592528450_1,3,15,57,0,0.4774791,2.329245,-1.7767653,496,0.5570642,1486
4,Coupled,model_6592528450_1,4,12,42,0,0.5218977,2.325269,-1.4100751,496,0.4901174,1718
5,Coupled,model_6592528450_1,5,15,46,2,0.5070372,2.358500,-1.3223362,496,0.5357312,1508
6,Coupled,model_6592528450_1,6,20,51,0,0.4869756,2.249658,-1.4796068,496,0.5706765,1198
7,Coupled,model_6592528450_1,7,19,63,0,0.4852614,2.230736,-1.8959978,496,0.5582201,1174
8,Coupled,model_6592528450_1,8,16,52,1,0.4792956,2.216584,-1.3405258,496,0.5472919,1398
9,Coupled,model_6592528450_1,9,9,23,1,0.4849891,1.970430,-1.5390336,496,0.5619010,1872


In [9]:
#1) Performance results
muTab<-aggregate(fast_data$Score, by=list(fast_data$Agent,fast_data$GameNb), FUN=mean)
colnames(muTab)<-c("Agent","GameNb","Score")
H_vec <- c(1:15)
for (i in 1:15)
{
    H_vec[i] <- muTab[which(muTab$Agent=="Streamed_Strategy" & muTab$GameNb==i),]$Score
}
M_vec <- c(1:15)
for (i in 1:15)
{
    M_vec[i] <- muTab[which(muTab$Agent=="Streamed" & muTab$GameNb==i),]$Score
}

#RSS per condition
rss_perf <- RSS(M_vec,H_vec)
rss_perf

print("MS17 comparison")
M2_vec <- c(1:15)
for (i in 1:15)
{
    M2_vec[i] <- muTab[which(muTab$Agent=="MS17" & muTab$GameNb==i),]$Score
}

rss2_perf <- RSS(M2_vec,H_vec)
rss2_perf

print("Blending comparison")
M3_vec <- c(1:15)
for (i in 1:15)
{
    M3_vec[i] <- muTab[which(muTab$Agent=="Coupled" & muTab$GameNb==i),]$Score
}

rss3_perf <- RSS(M3_vec,H_vec)
rss3_perf

[1] 128537.4

[1] "MS17 comparison"


[1] 30596710

[1] "Blending comparison"


[1] 16839949

In [10]:
#2) Entropy
fast_data <- data %>% filter(!Agent=='Coupled_Strategy')
fast_data <- fast_data %>% filter(!Entropy=='NaN')
muTab<-aggregate(fast_data$Entropy, by=list(fast_data$Agent,fast_data$GameNb), FUN=mean)

colnames(muTab)<-c("Agent","GameNb","Entropy")
H_vec <- c(1:15)
for (i in 1:15)
{
    H_vec[i] <- muTab[which(muTab$Agent=="Streamed_Strategy" & muTab$GameNb==i),]$Entropy
}
M_vec <- c(1:15)
for (i in 1:15)
{
    M_vec[i] <- muTab[which(muTab$Agent=="Streamed" & muTab$GameNb==i),]$Entropy
}

#RSS per condition
rss_ent <- RSS(M_vec,H_vec)
rss_ent

print("MS17 comparison")
M2_vec <- c(1:15)
for (i in 1:15)
{
    M2_vec[i] <- muTab[which(muTab$Agent=="MS17" & muTab$GameNb==i),]$Entropy
}

rss2_ent <- RSS(M2_vec,H_vec)
rss2_ent

print("Blending comparison")
M3_vec <- c(1:15)
for (i in 1:15)
{
    M3_vec[i] <- muTab[which(muTab$Agent=="Coupled" & muTab$GameNb==i),]$Entropy
}

rss3_ent <- RSS(M3_vec,H_vec)
rss3_ent

[1] 0.03522829

[1] "MS17 comparison"


[1] 1.729305

[1] "Blending comparison"


[1] 0.424036

In [11]:
#3) Log CV ISI
fast_data <- data %>% filter(!Agent=='Coupled_Strategy')
fast_data <- fast_data %>% filter(!LogCV=='NaN')
muTab<-aggregate(fast_data$LogCV, by=list(fast_data$Agent,fast_data$GameNb), FUN=mean)

colnames(muTab)<-c("Agent","GameNb","LogCV")
H_vec <- c(1:15)
for (i in 1:15)
{
    H_vec[i] <- muTab[which(muTab$Agent=="Streamed_Strategy" & muTab$GameNb==i),]$LogCV
}
M_vec <- c(1:15)
for (i in 1:15)
{
    M_vec[i] <- muTab[which(muTab$Agent=="Streamed" & muTab$GameNb==i),]$LogCV
}

#RSS per condition
rss_lcv <- RSS(M_vec,H_vec)
rss_lcv

print("MS17 comparison")
M2_vec <- c(1:15)
for (i in 1:15)
{
    M2_vec[i] <- muTab[which(muTab$Agent=="MS17" & muTab$GameNb==i),]$LogCV
}

rss2_lcv <- RSS(M2_vec,H_vec)
rss2_lcv

print("Blending comparison")
M3_vec <- c(1:15)
for (i in 1:15)
{
    M3_vec[i] <- muTab[which(muTab$Agent=="Coupled" & muTab$GameNb==i),]$LogCV
}

rss3_lcv <- RSS(M3_vec,H_vec)
rss3_lcv

[1] 0.1765483

[1] "MS17 comparison"


[1] 4.140125

[1] "Blending comparison"


[1] 0.3382848

In [12]:
#4) Resets
fast_data <- data %>% filter(!Agent=='Coupled_Strategy')
#fast_data <- fast_data %>% filter(!LogCV=='NaN')
muTab<-aggregate(fast_data$Resets, by=list(fast_data$Agent,fast_data$GameNb), FUN=mean)

colnames(muTab)<-c("Agent","GameNb","Resets")
H_vec <- c(1:15)
for (i in 1:15)
{
    H_vec[i] <- muTab[which(muTab$Agent=="Streamed_Strategy" & muTab$GameNb==i),]$Resets
}
M_vec <- c(1:15)
for (i in 1:15)
{
    M_vec[i] <- muTab[which(muTab$Agent=="Streamed" & muTab$GameNb==i),]$Resets
}

#RSS per condition
rss_res <- RSS(M_vec,H_vec)
rss_res

print("MS17 comparison")
M2_vec <- c(1:15)
for (i in 1:15)
{
    M2_vec[i] <- muTab[which(muTab$Agent=="MS17" & muTab$GameNb==i),]$Resets
}

rss2_res <- RSS(M2_vec,H_vec)
rss2_res

print("Blending comparison")
M3_vec <- c(1:15)
for (i in 1:15)
{
    M3_vec[i] <- muTab[which(muTab$Agent=="Coupled" & muTab$GameNb==i),]$Resets
}

rss3_res <- RSS(M3_vec,H_vec)
rss3_res

[1] 236.2458

[1] "MS17 comparison"


[1] 1464.731

[1] "Blending comparison"


[1] 552.1776

In [13]:
#5) Deflations
fast_data <- data %>% filter(!Agent=='Coupled_Strategy')
#fast_data <- fast_data %>% filter(!LogCV=='NaN')
muTab<-aggregate(fast_data$Deflations, by=list(fast_data$Agent,fast_data$GameNb), FUN=mean)

colnames(muTab)<-c("Agent","GameNb","Deflations")
H_vec <- c(1:15)
for (i in 1:15)
{
    H_vec[i] <- muTab[which(muTab$Agent=="Streamed_Strategy" & muTab$GameNb==i),]$Deflations
}
M_vec <- c(1:15)
for (i in 1:15)
{
    M_vec[i] <- muTab[which(muTab$Agent=="Streamed" & muTab$GameNb==i),]$Deflations
}

#RSS per condition
rss_def <- RSS(M_vec,H_vec)
rss_def

print("MS17 comparison")
M2_vec <- c(1:15)
for (i in 1:15)
{
    M2_vec[i] <- muTab[which(muTab$Agent=="MS17" & muTab$GameNb==i),]$Deflations
}

rss2_def <- RSS(M2_vec,H_vec)
rss2_def

print("Blending comparison")
M3_vec <- c(1:15)
for (i in 1:15)
{
    M3_vec[i] <- muTab[which(muTab$Agent=="Coupled" & muTab$GameNb==i),]$Deflations
}

rss3_def <- RSS(M3_vec,H_vec)
rss3_def

[1] 904.5399

[1] "MS17 comparison"


[1] 23838

[1] "Blending comparison"


[1] 6304.328

In [14]:
#6) Misses
fast_data <- data %>% filter(!Agent=='Coupled_Strategy')
#fast_data <- fast_data %>% filter(!LogCV=='NaN')
muTab<-aggregate(fast_data$Misses, by=list(fast_data$Agent,fast_data$GameNb), FUN=mean)

colnames(muTab)<-c("Agent","GameNb","Misses")
H_vec <- c(1:15)
for (i in 1:15)
{
    H_vec[i] <- muTab[which(muTab$Agent=="Streamed_Strategy" & muTab$GameNb==i),]$Misses
}
M_vec <- c(1:15)
for (i in 1:15)
{
    M_vec[i] <- muTab[which(muTab$Agent=="Streamed" & muTab$GameNb==i),]$Misses
}

#RSS per condition
rss_mis <- RSS(M_vec,H_vec)
rss_mis

print("MS17 comparison")
M2_vec <- c(1:15)
for (i in 1:15)
{
    M2_vec[i] <- muTab[which(muTab$Agent=="MS17" & muTab$GameNb==i),]$Misses
}

rss2_mis <- RSS(M2_vec,H_vec)
rss2_mis

print("Blending comparison")
M3_vec <- c(1:15)
for (i in 1:15)
{
    M3_vec[i] <- muTab[which(muTab$Agent=="Coupled" & muTab$GameNb==i),]$Misses
}

rss3_mis <- RSS(M3_vec,H_vec)
rss3_mis

[1] 2028.954

[1] "MS17 comparison"


[1] 6710.442

[1] "Blending comparison"


[1] 4113.064

In [15]:
#7) Shot periodicity
fast_data <- data %>% filter(!Agent=='Coupled_Strategy')
fast_data <- fast_data %>% filter(!ShotPer=='NaN')
muTab<-aggregate(fast_data$ShotPer, by=list(fast_data$Agent,fast_data$GameNb), FUN=mean)

colnames(muTab)<-c("Agent","GameNb","ShotPer")
H_vec <- c(1:15)
for (i in 1:15)
{
    H_vec[i] <- muTab[which(muTab$Agent=="Streamed_Strategy" & muTab$GameNb==i),]$ShotPer
}
M_vec <- c(1:15)
for (i in 1:15)
{
    M_vec[i] <- muTab[which(muTab$Agent=="Streamed" & muTab$GameNb==i),]$ShotPer
}

#RSS per condition
rss_per <- RSS(M_vec,H_vec)
rss_per

print("MS17 comparison")
M2_vec <- c(1:15)
for (i in 1:15)
{
    M2_vec[i] <- muTab[which(muTab$Agent=="MS17" & muTab$GameNb==i),]$ShotPer
}

rss2_per <- RSS(M2_vec,H_vec)
rss2_per

print("Blending comparison")
M3_vec <- c(1:15)
for (i in 1:15)
{
    M3_vec[i] <- muTab[which(muTab$Agent=="Coupled" & muTab$GameNb==i),]$ShotPer
}

rss3_per <- RSS(M3_vec,H_vec)
rss3_per

[1] 11279.69

[1] "MS17 comparison"


[1] 24434.85

[1] "Blending comparison"


[1] 158190.4

In [16]:
#8) Shot regularity
fast_data <- data %>% filter(!Agent=='Coupled_Strategy')
fast_data <- fast_data %>% filter(!ShotReg=='NaN')
muTab<-aggregate(fast_data$ShotReg, by=list(fast_data$Agent,fast_data$GameNb), FUN=mean)

colnames(muTab)<-c("Agent","GameNb","ShotReg")
H_vec <- c(1:15)
for (i in 1:15)
{
    H_vec[i] <- muTab[which(muTab$Agent=="Streamed_Strategy" & muTab$GameNb==i),]$ShotReg
}
M_vec <- c(1:15)
for (i in 1:15)
{
    M_vec[i] <- muTab[which(muTab$Agent=="Streamed" & muTab$GameNb==i),]$ShotReg
}

#RSS per condition
rss_reg <- RSS(M_vec,H_vec)
rss_reg

print("MS17 comparison")
M2_vec <- c(1:15)
for (i in 1:15)
{
    M2_vec[i] <- muTab[which(muTab$Agent=="MS17" & muTab$GameNb==i),]$ShotReg
}

rss2_reg <- RSS(M2_vec,H_vec)
rss2_reg

print("Blending comparison")
M3_vec <- c(1:15)
for (i in 1:15)
{
    M3_vec[i] <- muTab[which(muTab$Agent=="Coupled" & muTab$GameNb==i),]$ShotReg
}

rss3_reg <- RSS(M3_vec,H_vec)
rss3_reg

[1] 0.04677005

[1] "MS17 comparison"


[1] 0.7616653

[1] "Blending comparison"


[1] 0.03571712

In [17]:
#9) Right regularity
fast_data <- data %>% filter(!Agent=='Coupled_Strategy')
fast_data <- fast_data %>% filter(!RightReg=='NaN')
muTab<-aggregate(fast_data$RightReg, by=list(fast_data$Agent,fast_data$GameNb), FUN=mean)

colnames(muTab)<-c("Agent","GameNb","RightReg")
H_vec <- c(1:15)
for (i in 1:15)
{
    H_vec[i] <- muTab[which(muTab$Agent=="Streamed_Strategy" & muTab$GameNb==i),]$RightReg
}
M_vec <- c(1:15)
for (i in 1:15)
{
    M_vec[i] <- muTab[which(muTab$Agent=="Streamed" & muTab$GameNb==i),]$RightReg
}

#RSS per condition
rss_Rreg <- RSS(M_vec,H_vec)
rss_Rreg

print("MS17 comparison")
M2_vec <- c(1:15)
for (i in 1:15)
{
    M2_vec[i] <- muTab[which(muTab$Agent=="MS17" & muTab$GameNb==i),]$RightReg
}

rss2_Rreg <- RSS(M2_vec,H_vec)
rss2_Rreg

print("Blending comparison")
M3_vec <- c(1:15)
for (i in 1:15)
{
    M3_vec[i] <- muTab[which(muTab$Agent=="Coupled" & muTab$GameNb==i),]$RightReg
}

rss3_Rreg <- RSS(M3_vec,H_vec)
rss3_Rreg

[1] 0.006619956

[1] "MS17 comparison"


[1] 0.01918736

[1] "Blending comparison"


[1] 0.8454717

In [18]:
#BIC measure - Fast model
n=15
k=0
Points_BIC <- BIC(rss_perf,n,k)
Points_BIC
Res_BIC <- BIC(rss_res,n,k)
Res_BIC
Def_BIC <- BIC(rss_def,n,k)
Def_BIC
Mis_BIC <- BIC(rss_mis,n,k)
Mis_BIC
print("variability")
Ent_BIC <- BIC(rss_ent,n,k)
Ent_BIC
lCV_BIC <- BIC(rss_lcv,n,k)
lCV_BIC
print("ACF measures")
Per_BIC <- BIC(rss_per,n,k)
Per_BIC
Reg_BIC <- BIC(rss_reg,n,k)
Reg_BIC
RightReg_BIC <- BIC(rss_Rreg,n,k)
RightReg_BIC

[1] 135.8389

[1] 41.35234

[1] 61.49064

[1] 73.60838

[1] "variability"


...

[1] -66.63317

[1] "ACF measures"


[1] 99.34064

[1] -86.55844

[1] -115.8858

In [19]:
#AIC measure - Fast model
Points_AIC <- AIC(rss_perf,n,k)
Points_AIC
Res_AIC <- AIC(rss_res,n,k)
Res_AIC
Def_AIC <- AIC(rss_def,n,k)
Def_AIC
Mis_AIC <- AIC(rss_mis,n,k)
Mis_AIC
print("Variability")
Ent_AIC <- AIC(rss_ent,n,k)
Ent_AIC
lCV_AIC <- AIC(rss_lcv,n,k)
lCV_AIC
print("Autocorrelation")
Per_AIC <- AIC(rss_per,n,k)
Per_AIC
Reg_AIC <- AIC(rss_reg,n,k)
Reg_AIC
RightReg_AIC <- AIC(rss_Rreg,n,k)
RightReg_AIC

[1] 176.4596

[1] 81.97309

[1] 102.1114

[1] 114.2291

[1] "Variability"


...

[1] -26.01241

[1] "Autocorrelation"


[1] 139.9614

[1] -45.93768

[1] -75.265

In [20]:
#BIC measure - MS17 model
n=15
k=0
Points_BIC <- BIC(rss2_perf,n,k)
Points_BIC
Res_BIC <- BIC(rss2_res,n,k)
Res_BIC
Def_BIC <- BIC(rss2_def,n,k)
Def_BIC
Mis_BIC <- BIC(rss2_mis,n,k)
Mis_BIC
print("variability")
Ent_BIC <- BIC(rss2_ent,n,k)
Ent_BIC
lCV_BIC <- BIC(rss2_lcv,n,k)
lCV_BIC
print("ACF measures")
Per_BIC <- BIC(rss2_per,n,k)
Per_BIC
Reg_BIC <- BIC(rss2_reg,n,k)
Reg_BIC
RightReg_BIC <- BIC(rss2_Rreg,n,k)
RightReg_BIC

[1] 217.9253

[1] 68.72065

[1] 110.5648

[1] 91.55055

[1] "variability"


...

[1] -19.30986

[1] "ACF measures"


[1] 110.9357

[1] -44.70447

[1] -99.92331

In [21]:
#AIC measure - MS17 model
Points_AIC <- AIC(rss2_perf,n,k)
Points_AIC
Res_AIC <- AIC(rss2_res,n,k)
Res_AIC
Def_AIC <- AIC(rss2_def,n,k)
Def_AIC
Mis_AIC <- AIC(rss2_mis,n,k)
Mis_AIC
print("Variability")
Ent_AIC <- AIC(rss2_ent,n,k)
Ent_AIC
lCV_AIC <- AIC(rss2_lcv,n,k)
lCV_AIC
print("Autocorrelation")
Per_AIC <- AIC(rss2_per,n,k)
Per_AIC
Reg_AIC <- AIC(rss2_reg,n,k)
Reg_AIC
RightReg_AIC <- AIC(rss2_Rreg,n,k)
RightReg_AIC

[1] 258.546

[1] 109.3414

[1] 151.1855

[1] 132.1713

[1] "Variability"


...

[1] 21.31089

[1] "Autocorrelation"


[1] 151.5565

[1] -4.083721

[1] -59.30255

In [22]:
#BIC measure - Blending model
n=15
k=0
Points_BIC <- BIC(rss3_perf,n,k)
Points_BIC
Res_BIC <- BIC(rss3_res,n,k)
Res_BIC
Def_BIC <- BIC(rss3_def,n,k)
Def_BIC
Mis_BIC <- BIC(rss3_mis,n,k)
Mis_BIC
print("variability")
Ent_BIC <- BIC(rss3_ent,n,k)
Ent_BIC
lCV_BIC <- BIC(rss3_lcv,n,k)
lCV_BIC
print("ACF measures")
Per_BIC <- BIC(rss3_per,n,k)
Per_BIC
Reg_BIC <- BIC(rss3_reg,n,k)
Reg_BIC
RightReg_BIC <- BIC(rss3_Rreg,n,k)
RightReg_BIC

[1] 208.9682

[1] 54.08729

[1] 90.61412

[1] 84.2081

[1] "variability"


...

[1] -56.87876

[1] "ACF measures"


[1] 138.9526

[1] -90.60263

[1] -43.13866

In [23]:
#AIC measure - Blending model
Points_AIC <- AIC(rss3_perf,n,k)
Points_AIC
Res_AIC <- AIC(rss3_res,n,k)
Res_AIC
Def_AIC <- AIC(rss3_def,n,k)
Def_AIC
Mis_AIC <- AIC(rss3_mis,n,k)
Mis_AIC
print("Variability")
Ent_AIC <- AIC(rss3_ent,n,k)
Ent_AIC
lCV_AIC <- AIC(rss3_lcv,n,k)
lCV_AIC
print("Autocorrelation")
Per_AIC <- AIC(rss3_per,n,k)
Per_AIC
Reg_AIC <- AIC(rss3_reg,n,k)
Reg_AIC
RightReg_AIC <- AIC(rss3_Rreg,n,k)
RightReg_AIC

[1] 249.589

[1] 94.70805

[1] 131.2349

[1] 124.8289

[1] "Variability"


...

[1] -16.25801

[1] "Autocorrelation"


[1] 179.5733

[1] -49.98188

[1] -2.517909

# Compute BIC and AIC slow-shooting model

In [24]:
#Filtered data set
slow_data <- data %>% filter(!Agent=='Streamed_Strategy')
head(slow_data,10)

,Agent,subject,GameNb,Deflations,Misses,Resets,ShotReg,Entropy,LogCV,ShotPer,RightReg,Score
,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,Coupled,model_6592528450_1,1,96,63,3,0.2749406,2.938219,-0.9013436,480,0.1648576,256
2,Coupled,model_6592528450_1,2,22,107,2,0.4118143,2.848877,-1.3099846,512,0.3605711,888
3,Coupled,model_6592528450_1,3,15,57,0,0.4774791,2.329245,-1.7767653,496,0.5570642,1486
4,Coupled,model_6592528450_1,4,12,42,0,0.5218977,2.325269,-1.4100751,496,0.4901174,1718
5,Coupled,model_6592528450_1,5,15,46,2,0.5070372,2.358500,-1.3223362,496,0.5357312,1508
6,Coupled,model_6592528450_1,6,20,51,0,0.4869756,2.249658,-1.4796068,496,0.5706765,1198
7,Coupled,model_6592528450_1,7,19,63,0,0.4852614,2.230736,-1.8959978,496,0.5582201,1174
8,Coupled,model_6592528450_1,8,16,52,1,0.4792956,2.216584,-1.3405258,496,0.5472919,1398
9,Coupled,model_6592528450_1,9,9,23,1,0.4849891,1.970430,-1.5390336,496,0.5619010,1872


In [25]:
#1) Performance results
muTab<-aggregate(slow_data$Score, by=list(slow_data$Agent,slow_data$GameNb), FUN=mean)
colnames(muTab)<-c("Agent","GameNb","Score")
H_vec <- c(1:15)
for (i in 1:15)
{
    H_vec[i] <- muTab[which(muTab$Agent=="Coupled_Strategy" & muTab$GameNb==i),]$Score
}
M_vec <- c(1:15)
for (i in 1:15)
{
    M_vec[i] <- muTab[which(muTab$Agent=="Coupled" & muTab$GameNb==i),]$Score
}

#RSS per condition
rss_perf <- RSS(M_vec,H_vec)
rss_perf

print("MS17 comparison")
M2_vec <- c(1:15)
for (i in 1:15)
{
    M2_vec[i] <- muTab[which(muTab$Agent=="MS17" & muTab$GameNb==i),]$Score
}

rss2_perf <- RSS(M2_vec,H_vec)
rss2_perf

print("Hybrid comparison")
M3_vec <- c(1:15)
for (i in 1:15)
{
    M3_vec[i] <- muTab[which(muTab$Agent=="Streamed" & muTab$GameNb==i),]$Score
}

rss3_perf <- RSS(M3_vec,H_vec)
rss3_perf

[1] 258265.2

[1] "MS17 comparison"


[1] 1206358

[1] "Hybrid comparison"


[1] 21390836

In [26]:
#2) Entropy
slow_data <- data %>% filter(!Agent=='Streamed_Strategy')
slow_data <- slow_data %>% filter(!Entropy=='NaN')
muTab<-aggregate(slow_data$Entropy, by=list(slow_data$Agent,slow_data$GameNb), FUN=mean)

colnames(muTab)<-c("Agent","GameNb","Entropy")
H_vec <- c(1:15)
for (i in 1:15)
{
    H_vec[i] <- muTab[which(muTab$Agent=="Coupled_Strategy" & muTab$GameNb==i),]$Entropy
}
M_vec <- c(1:15)
for (i in 1:15)
{
    M_vec[i] <- muTab[which(muTab$Agent=="Coupled" & muTab$GameNb==i),]$Entropy
}

#RSS per condition
rss_ent <- RSS(M_vec,H_vec)
rss_ent

print("MS17 comparison")
M2_vec <- c(1:15)
for (i in 1:15)
{
    M2_vec[i] <- muTab[which(muTab$Agent=="MS17" & muTab$GameNb==i),]$Entropy
}

rss2_ent <- RSS(M2_vec,H_vec)
rss2_ent

print("Hybrid comparison")
M3_vec <- c(1:15)
for (i in 1:15)
{
    M3_vec[i] <- muTab[which(muTab$Agent=="Streamed" & muTab$GameNb==i),]$Entropy
}

rss3_ent <- RSS(M3_vec,H_vec)
rss3_ent

[1] 0.5293885

[1] "MS17 comparison"


[1] 1.307697

[1] "Hybrid comparison"


[1] 0.154259

In [27]:
#3) Log CV ISI
slow_data <- data %>% filter(!Agent=='Streamed_Strategy')
slow_data <- slow_data %>% filter(!LogCV=='NaN')
muTab<-aggregate(slow_data$LogCV, by=list(slow_data$Agent,slow_data$GameNb), FUN=mean)

colnames(muTab)<-c("Agent","GameNb","LogCV")
H_vec <- c(1:15)
for (i in 1:15)
{
    H_vec[i] <- muTab[which(muTab$Agent=="Coupled_Strategy" & muTab$GameNb==i),]$LogCV
}
M_vec <- c(1:15)
for (i in 1:15)
{
    M_vec[i] <- muTab[which(muTab$Agent=="Coupled" & muTab$GameNb==i),]$LogCV
}

#RSS per condition
rss_lcv <- RSS(M_vec,H_vec)
rss_lcv

print("MS17 comparison")
M2_vec <- c(1:15)
for (i in 1:15)
{
    M2_vec[i] <- muTab[which(muTab$Agent=="MS17" & muTab$GameNb==i),]$LogCV
}

rss2_lcv <- RSS(M2_vec,H_vec)
rss2_lcv

print("Hybrid comparison")
M3_vec <- c(1:15)
for (i in 1:15)
{
    M3_vec[i] <- muTab[which(muTab$Agent=="Streamed" & muTab$GameNb==i),]$LogCV
}

rss3_lcv <- RSS(M3_vec,H_vec)
rss3_lcv

[1] 0.1699786

[1] "MS17 comparison"


[1] 1.53027

[1] "Hybrid comparison"


[1] 0.3616087

In [28]:
#4) Resets
slow_data <- data %>% filter(!Agent=='Streamed_Strategy')
muTab<-aggregate(slow_data$Resets, by=list(slow_data$Agent,slow_data$GameNb), FUN=mean)

colnames(muTab)<-c("Agent","GameNb","Resets")
H_vec <- c(1:15)
for (i in 1:15)
{
    H_vec[i] <- muTab[which(muTab$Agent=="Coupled_Strategy" & muTab$GameNb==i),]$Resets
}
M_vec <- c(1:15)
for (i in 1:15)
{
    M_vec[i] <- muTab[which(muTab$Agent=="Coupled" & muTab$GameNb==i),]$Resets
}

#RSS per condition
rss_res <- RSS(M_vec,H_vec)
rss_res

print("MS17 comparison")
M2_vec <- c(1:15)
for (i in 1:15)
{
    M2_vec[i] <- muTab[which(muTab$Agent=="MS17" & muTab$GameNb==i),]$Resets
}

rss2_res <- RSS(M2_vec,H_vec)
rss2_res

print("Hybrid comparison")
M3_vec <- c(1:15)
for (i in 1:15)
{
    M3_vec[i] <- muTab[which(muTab$Agent=="Streamed" & muTab$GameNb==i),]$Resets
}

rss3_res <- RSS(M3_vec,H_vec)
rss3_res

[1] 953.1858

[1] "MS17 comparison"


[1] 1282.892

[1] "Hybrid comparison"


[1] 465.7168

In [29]:
#5) Deflations
slow_data <- data %>% filter(!Agent=='Streamed_Strategy')
muTab<-aggregate(slow_data$Deflations, by=list(slow_data$Agent,slow_data$GameNb), FUN=mean)

colnames(muTab)<-c("Agent","GameNb","Deflations")
H_vec <- c(1:15)
for (i in 1:15)
{
    H_vec[i] <- muTab[which(muTab$Agent=="Coupled_Strategy" & muTab$GameNb==i),]$Deflations
}
M_vec <- c(1:15)
for (i in 1:15)
{
    M_vec[i] <- muTab[which(muTab$Agent=="Coupled" & muTab$GameNb==i),]$Deflations
}

#RSS per condition
rss_def <- RSS(M_vec,H_vec)
rss_def

print("MS17 comparison")
M2_vec <- c(1:15)
for (i in 1:15)
{
    M2_vec[i] <- muTab[which(muTab$Agent=="MS17" & muTab$GameNb==i),]$Deflations
}

rss2_def <- RSS(M2_vec,H_vec)
rss2_def

print("Hybrid comparison")
M3_vec <- c(1:15)
for (i in 1:15)
{
    M3_vec[i] <- muTab[which(muTab$Agent=="Streamed" & muTab$GameNb==i),]$Deflations
}

rss3_def <- RSS(M3_vec,H_vec)
rss3_def

[1] 2331.083

[1] "MS17 comparison"


[1] 9436.143

[1] "Hybrid comparison"


[1] 4540.705

In [30]:
#6) Misses
slow_data <- data %>% filter(!Agent=='Streamed_Strategy')
muTab<-aggregate(slow_data$Misses, by=list(slow_data$Agent,slow_data$GameNb), FUN=mean)

colnames(muTab)<-c("Agent","GameNb","Misses")
H_vec <- c(1:15)
for (i in 1:15)
{
    H_vec[i] <- muTab[which(muTab$Agent=="Coupled_Strategy" & muTab$GameNb==i),]$Misses
}
M_vec <- c(1:15)
for (i in 1:15)
{
    M_vec[i] <- muTab[which(muTab$Agent=="Coupled" & muTab$GameNb==i),]$Misses
}

#RSS per condition
rss_mis <- RSS(M_vec,H_vec)
rss_mis

print("MS17 comparison")
M2_vec <- c(1:15)
for (i in 1:15)
{
    M2_vec[i] <- muTab[which(muTab$Agent=="MS17" & muTab$GameNb==i),]$Misses
}

rss2_mis <- RSS(M2_vec,H_vec)
rss2_mis

print("Hybrid comparison")
M3_vec <- c(1:15)
for (i in 1:15)
{
    M3_vec[i] <- muTab[which(muTab$Agent=="Streamed" & muTab$GameNb==i),]$Misses
}

rss3_mis <- RSS(M3_vec,H_vec)
rss3_mis

[1] 1927.205

[1] "MS17 comparison"


[1] 27420.89

[1] "Hybrid comparison"


[1] 3124.126

In [31]:
#7) Shot periodicity
slow_data <- data %>% filter(!Agent=='Streamed_Strategy')
slow_data <- slow_data %>% filter(!ShotPer=='NaN')
muTab<-aggregate(slow_data$ShotPer, by=list(slow_data$Agent,slow_data$GameNb), FUN=mean)

colnames(muTab)<-c("Agent","GameNb","ShotPer")
H_vec <- c(1:15)
for (i in 1:15)
{
    H_vec[i] <- muTab[which(muTab$Agent=="Coupled_Strategy" & muTab$GameNb==i),]$ShotPer
}
M_vec <- c(1:15)
for (i in 1:15)
{
    M_vec[i] <- muTab[which(muTab$Agent=="Coupled" & muTab$GameNb==i),]$ShotPer
}

#RSS per condition
rss_per <- RSS(M_vec,H_vec)
rss_per

print("MS17 comparison")
M2_vec <- c(1:15)
for (i in 1:15)
{
    M2_vec[i] <- muTab[which(muTab$Agent=="MS17" & muTab$GameNb==i),]$ShotPer
}

rss2_per <- RSS(M2_vec,H_vec)
rss2_per

print("Hybrid comparison")
M3_vec <- c(1:15)
for (i in 1:15)
{
    M3_vec[i] <- muTab[which(muTab$Agent=="Streamed" & muTab$GameNb==i),]$ShotPer
}

rss3_per <- RSS(M3_vec,H_vec)
rss3_per

[1] 1386.115

[1] "MS17 comparison"


[1] 78021.67

[1] "Hybrid comparison"


[1] 226558.8

In [32]:
#8) Shot regularity
slow_data <- data %>% filter(!Agent=='Streamed_Strategy')
slow_data <- slow_data %>% filter(!ShotReg=='NaN')
muTab<-aggregate(slow_data$ShotReg, by=list(slow_data$Agent,slow_data$GameNb), FUN=mean)

colnames(muTab)<-c("Agent","GameNb","ShotReg")
H_vec <- c(1:15)
for (i in 1:15)
{
    H_vec[i] <- muTab[which(muTab$Agent=="Coupled_Strategy" & muTab$GameNb==i),]$ShotReg
}
M_vec <- c(1:15)
for (i in 1:15)
{
    M_vec[i] <- muTab[which(muTab$Agent=="Coupled" & muTab$GameNb==i),]$ShotReg
}

#RSS per condition
rss_reg <- RSS(M_vec,H_vec)
rss_reg

print("MS17 comparison")
M2_vec <- c(1:15)
for (i in 1:15)
{
    M2_vec[i] <- muTab[which(muTab$Agent=="MS17" & muTab$GameNb==i),]$ShotReg
}

rss2_reg <- RSS(M2_vec,H_vec)
rss2_reg

print("Hybrid comparison")
M3_vec <- c(1:15)
for (i in 1:15)
{
    M3_vec[i] <- muTab[which(muTab$Agent=="Streamed" & muTab$GameNb==i),]$ShotReg
}

rss3_reg <- RSS(M3_vec,H_vec)
rss3_reg

[1] 0.04578315

[1] "MS17 comparison"


[1] 0.7858907

[1] "Hybrid comparison"


[1] 0.04809708

In [33]:
#9) Right regularity
slow_data <- data %>% filter(!Agent=='Streamed_Strategy')
slow_data <- slow_data %>% filter(!RightReg=='NaN')
muTab<-aggregate(slow_data$RightReg, by=list(slow_data$Agent,slow_data$GameNb), FUN=mean)

colnames(muTab)<-c("Agent","GameNb","RightReg")
H_vec <- c(1:15)
for (i in 1:15)
{
    H_vec[i] <- muTab[which(muTab$Agent=="Coupled_Strategy" & muTab$GameNb==i),]$RightReg
}
M_vec <- c(1:15)
for (i in 1:15)
{
    M_vec[i] <- muTab[which(muTab$Agent=="Coupled" & muTab$GameNb==i),]$RightReg
}

#RSS per condition
rss_Rreg <- RSS(M_vec,H_vec)
rss_Rreg

print("MS17 comparison")
M2_vec <- c(1:15)
for (i in 1:15)
{
    M2_vec[i] <- muTab[which(muTab$Agent=="MS17" & muTab$GameNb==i),]$RightReg
}

rss2_Rreg <- RSS(M2_vec,H_vec)
rss2_Rreg

print("Hybrid comparison")
M3_vec <- c(1:15)
for (i in 1:15)
{
    M3_vec[i] <- muTab[which(muTab$Agent=="Streamed" & muTab$GameNb==i),]$RightReg
}

rss3_Rreg <- RSS(M3_vec,H_vec)
rss3_Rreg

[1] 0.07295036

[1] "MS17 comparison"


[1] 0.3774491

[1] "Hybrid comparison"


[1] 0.455122

In [34]:
#BIC measure - Slow model
n=15
k=0
Points_BIC <- BIC(rss_perf,n,k)
Points_BIC
Res_BIC <- BIC(rss_res,n,k)
Res_BIC
Def_BIC <- BIC(rss_def,n,k)
Def_BIC
Mis_BIC <- BIC(rss_mis,n,k)
Mis_BIC
print("variability")
Ent_BIC <- BIC(rss_ent,n,k)
Ent_BIC
lCV_BIC <- BIC(rss_lcv,n,k)
lCV_BIC
print("ACF measures")
Per_BIC <- BIC(rss_per,n,k)
Per_BIC
Reg_BIC <- BIC(rss_reg,n,k)
Reg_BIC
RightReg_BIC <- BIC(rss_Rreg,n,k)
RightReg_BIC

[1] 146.3054

[1] 62.27639

[1] 75.69057

[1] 72.83664

[1] "variability"


...

[1] -67.202

[1] "ACF measures"


[1] 67.89315

[1] -86.87834

[1] -79.89039

In [35]:
#AIC measure - Slow model
Points_AIC <- AIC(rss_perf,n,k)
Points_AIC
Res_AIC <- AIC(rss_res,n,k)
Res_AIC
Def_AIC <- AIC(rss_def,n,k)
Def_AIC
Mis_AIC <- AIC(rss_mis,n,k)
Mis_AIC
print("Variability")
Ent_AIC <- AIC(rss_ent,n,k)
Ent_AIC
lCV_AIC <- AIC(rss_lcv,n,k)
lCV_AIC
print("Autocorrelation")
Per_AIC <- AIC(rss_per,n,k)
Per_AIC
Reg_AIC <- AIC(rss_reg,n,k)
Reg_AIC
RightReg_AIC <- AIC(rss_Rreg,n,k)
RightReg_AIC

[1] 186.9261

[1] 102.8971

[1] 116.3113

[1] 113.4574

[1] "Variability"


...

[1] -26.58124

[1] "Autocorrelation"


[1] 108.5139

[1] -46.25759

[1] -39.26964

In [36]:
#BIC measure - MS17 model
n=15
k=0
Points_BIC <- BIC(rss2_perf,n,k)
Points_BIC
Res_BIC <- BIC(rss2_res,n,k)
Res_BIC
Def_BIC <- BIC(rss2_def,n,k)
Def_BIC
Mis_BIC <- BIC(rss2_mis,n,k)
Mis_BIC
print("variability")
Ent_BIC <- BIC(rss2_ent,n,k)
Ent_BIC
lCV_BIC <- BIC(rss2_lcv,n,k)
lCV_BIC
print("ACF measures")
Per_BIC <- BIC(rss2_per,n,k)
Per_BIC
Reg_BIC <- BIC(rss2_reg,n,k)
Reg_BIC
RightReg_BIC <- BIC(rss2_Rreg,n,k)
RightReg_BIC

[1] 169.426

[1] 66.73233

[1] 96.66379

[1] 112.6652

[1] "variability"


...

[1] -34.23909

[1] "ACF measures"


[1] 128.3504

[1] -44.23482

[1] -55.23555

In [37]:
#AIC measure - MS17 model
Points_AIC <- AIC(rss2_perf,n,k)
Points_AIC
Res_AIC <- AIC(rss2_res,n,k)
Res_AIC
Def_AIC <- AIC(rss2_def,n,k)
Def_AIC
Mis_AIC <- AIC(rss2_mis,n,k)
Mis_AIC
print("Variability")
Ent_AIC <- AIC(rss2_ent,n,k)
Ent_AIC
lCV_AIC <- AIC(rss2_lcv,n,k)
lCV_AIC
print("Autocorrelation")
Per_AIC <- AIC(rss2_per,n,k)
Per_AIC
Reg_AIC <- AIC(rss2_reg,n,k)
Reg_AIC
RightReg_AIC <- AIC(rss2_Rreg,n,k)
RightReg_AIC

[1] 210.0467

[1] 107.3531

[1] 137.2845

[1] 153.2859

[1] "Variability"


...

[1] 6.381663

[1] "Autocorrelation"


[1] 168.9711

[1] -3.614064

[1] -14.61479

In [38]:
#BIC measure - Hybrid model
n=15
k=0
Points_BIC <- BIC(rss3_perf,n,k)
Points_BIC
Res_BIC <- BIC(rss3_res,n,k)
Res_BIC
Def_BIC <- BIC(rss3_def,n,k)
Def_BIC
Mis_BIC <- BIC(rss3_mis,n,k)
Mis_BIC
print("variability")
Ent_BIC <- BIC(rss3_ent,n,k)
Ent_BIC
lCV_BIC <- BIC(rss3_lcv,n,k)
lCV_BIC
print("ACF measures")
Per_BIC <- BIC(rss3_per,n,k)
Per_BIC
Reg_BIC <- BIC(rss3_reg,n,k)
Reg_BIC
RightReg_BIC <- BIC(rss3_Rreg,n,k)
RightReg_BIC

[1] 212.5563

[1] 51.53291

[1] 85.69181

[1] 80.08289

[1] "variability"


...

[1] -55.87864

[1] "ACF measures"


[1] 144.3406

[1] -86.13876

[1] -52.4286

In [39]:
#AIC measure - Hybrid model
Points_AIC <- AIC(rss3_perf,n,k)
Points_AIC
Res_AIC <- AIC(rss3_res,n,k)
Res_AIC
Def_AIC <- AIC(rss3_def,n,k)
Def_AIC
Mis_AIC <- AIC(rss3_mis,n,k)
Mis_AIC
print("Variability")
Ent_AIC <- AIC(rss3_ent,n,k)
Ent_AIC
lCV_AIC <- AIC(rss3_lcv,n,k)
lCV_AIC
print("Autocorrelation")
Per_AIC <- AIC(rss3_per,n,k)
Per_AIC
Reg_AIC <- AIC(rss3_reg,n,k)
Reg_AIC
RightReg_AIC <- AIC(rss3_Rreg,n,k)
RightReg_AIC

[1] 253.1771

[1] 92.15367

[1] 126.3126

[1] 120.7036

[1] "Variability"


...

[1] -15.25789

[1] "Autocorrelation"


[1] 184.9614

[1] -45.51801

[1] -11.80785